<h1>Experiment on bilateral adaptive weights - Middlebury 2014 - popduct</h1>
<h2>For results and discussion please visit&nbsp;<a href="ALG_006_EXP_005-Comparative_analysis-VIS.ipynb" target="_blank" rel="noopener">ALG_006_EXP_005-Comparative_analysis-VIS</a>.&nbsp;</h2>

In [14]:
import pandas as pd
import ipywidgets as widgets
import numpy as np
import sys
import os

sys.path.append(os.path.join("..", ".."))
import glob
import cv2

import plotly.graph_objs as go
import plotly.express as px
from ipywidgets import HBox, VBox, Button

from components.utils import plotly_helpers as ph
from components.utils import utils as u

In [15]:

available_metrix = ['abs_error',
       'mse', 'avg', 'eucledian', 'bad1', 'bad2', 'bad4', 'bad8']

metrics_selector = widgets.Dropdown(
    options=[(m,m) for m in available_metrix],
    description='Metric:',
    value="bad4"
)


nonoccluded = widgets.Dropdown(
    options=[("yes", False), ("No", True)],
    description='Nonoccluded:'
)


### Please select metrics and whether occlusions are counted as errors

In [16]:
VBox([metrics_selector, nonoccluded])

### Loading the data and building the plots

In [17]:

selected_file = os.path.join("..","..", "benchmarking", "MiddEval", "custom_log", "blg_mul_benchmarking.csv")
#selected_file = "./fixed_csv2.csv"
df = ph.load_n_clean(selected_file)

##Filtering to selected occlusion parameter

df = df[df["are_occlusions_errors"]==nonoccluded.value]
df.sort_values(by=["scene", "match", "h", "w"], inplace=True)

number_of_samples = df.shape[0]

In [18]:
### Dashboard 1


from ipywidgets import Image, Layout

img_widget = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))

fig_a = ph.get_figure_widget (df, "scene", metrics_selector.value, 
                           "Scene w.r.t."+metrics_selector.value)
fig_b = ph.get_figure_widget (df, "match", "kernel_size", "Kernel sizes w.r.t. match values")


figs = [fig_a, fig_b]
ph.bind_hover_function(figs, img_widget, df)
ph.bind_brush_function(figs, df)

button = ph.get_reset_brush_button(figs)
dashboard1 = VBox([button, fig_a,
                  HBox([img_widget, fig_b])])


### Dashboard 2

df.sort_values(by=["experiment_id"])
traced_fig_1, dfs_1 = ph.get_figure_widget_traced(df, "scene", metrics_selector.value, "experiment_id")

traced_fig_widget_1 = go.FigureWidget(traced_fig_1)



traced_fig_1_imw_1 = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))
traced_fig_1_imw_2 = Image(value=df["loaded_gts"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))

#figs, img_widget, selected_scene_df
ph.bind_hover_function2([traced_fig_widget_1], traced_fig_1_imw_1, dfs_1, img_widget_groundtruth=traced_fig_1_imw_2)


turn_the_lights_on = ph.get_dropdown_widget(["On", "Off"], label="Turn plots:", values = [True, False])

ph.bind_dropdown_switch_traces_fn(turn_the_lights_on, traced_fig_widget_1)

dashboard2 = VBox([turn_the_lights_on, traced_fig_widget_1, HBox([traced_fig_1_imw_1,traced_fig_1_imw_2])])


### Dashboard 3


traced_fig_2, dfs_2 = ph.get_figure_widget_traced(df, "experiment_id", metrics_selector.value, "scene")

traced_fig_widget_2 = go.FigureWidget(traced_fig_2)

traced_fig_2_imw_1 = Image(value=df["loaded_imgs"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))
traced_fig_2_imw_2 = Image(value=df["loaded_gts"].iloc[0], 
                   layout=Layout(height='375px', width='450px'))



#figs, img_widget, selected_scene_df
ph.bind_hover_function2([traced_fig_widget_2], traced_fig_2_imw_1, dfs_2, img_widget_groundtruth=traced_fig_2_imw_2)

turn_the_lights_on_2 = ph.get_dropdown_widget(["On", "Off"], label="Turn plots:", values = [True, False])

ph.bind_dropdown_switch_traces_fn(turn_the_lights_on_2, traced_fig_widget_2)


dashboard3 = VBox([turn_the_lights_on_2, traced_fig_widget_2, HBox([traced_fig_2_imw_1,traced_fig_2_imw_2])])


In [21]:
df.pivot_table(index = ["experiment_id", "kernel_size", "match"], values = "bad4", aggfunc=np.mean).sort_values(by="bad4").head(10)

,,,bad4
experiment_id,kernel_size,match,
blg_40_5x7gc_8_gs_90_alph_0,5x7,40,0.336915
blg_40_7x3gc_8_gs_90_alph_0,7x3,40,0.338746
blg_50_5x7gc_8_gs_90_alph_0,5x7,50,0.340378
blg_30_5x7gc_8_gs_90_alph_0,5x7,30,0.340635
blg_30_7x3gc_8_gs_90_alph_0,7x3,30,0.340899
blg_50_7x3gc_8_gs_90_alph_0,7x3,50,0.343219
blg_40_5x7gc_10_gs_90_alph_0,5x7,40,0.344960
blg_30_3x5gc_8_gs_90_alph_0,3x5,30,0.345930
blg_30_5x7gc_10_gs_90_alph_0,5x7,30,0.346022


### To aid interaction with the plots, the best results in a tabular form are displayed below

In [19]:
df.pivot_table(index = "experiment_id", columns="scene", values = "bad4", aggfunc=np.min).head(10)

scene,Adirondack,ArtL,Jadeplant,Motorcycle,MotorcycleE,Piano,PianoL,Pipes,Playroom,Playtable,PlaytableP,Recycle,Shelves,Teddy,Vintage
experiment_id,,,,,,,,,,,,,,,
blg_30_3x3gc_10_gs_90_alph_0,0.261458,0.270487,0.627612,0.169646,0.566725,0.368633,0.587173,0.286050,0.378244,0.439260,0.316700,0.283596,0.439047,0.199148,0.647518
blg_30_3x3gc_12_gs_90_alph_0,0.265639,0.287288,0.628197,0.167624,0.593938,0.373060,0.600297,0.286356,0.382165,0.440210,0.316396,0.282274,0.436584,0.201891,0.647592
blg_30_3x3gc_8_gs_90_alph_0,0.259984,0.259127,0.627832,0.170694,0.531452,0.362279,0.571545,0.284274,0.379899,0.444341,0.319813,0.283474,0.444494,0.207072,0.648216
blg_30_3x5gc_10_gs_90_alph_0,0.230617,0.259794,0.625643,0.151947,0.493925,0.354021,0.566210,0.256625,0.359482,0.389895,0.265905,0.242124,0.412965,0.135014,0.637357
blg_30_3x5gc_12_gs_90_alph_0,0.235463,0.268841,0.627402,0.151944,0.515470,0.358958,0.581630,0.258611,0.366548,0.392487,0.265249,0.251268,0.408533,0.134062,0.637395
blg_30_3x5gc_8_gs_90_alph_0,0.224483,0.251686,0.623872,0.153916,0.466221,0.347645,0.551474,0.253438,0.355764,0.392640,0.268728,0.230801,0.415798,0.136331,0.637602
blg_30_5x7gc_10_gs_90_alph_0,0.215061,0.251561,0.622317,0.151096,0.452959,0.348434,0.551924,0.254026,0.352895,0.368962,0.245474,0.219730,0.400055,0.129681,0.626155
blg_30_5x7gc_12_gs_90_alph_0,0.221384,0.256511,0.624609,0.150097,0.476166,0.353089,0.566642,0.255509,0.358529,0.373789,0.249580,0.233539,0.402243,0.131780,0.629336
blg_30_5x7gc_8_gs_90_alph_0,0.207248,0.245818,0.622559,0.152668,0.424168,0.340819,0.532229,0.252243,0.348962,0.368387,0.246033,0.211895,0.402259,0.129942,0.624292


### Dashboard 1: Scene w.r.t. {metric} (selection plot)
<ol>
    <li>The following figure allows to use the "lasso" tool as a tool of selection.</li>
    <li>As a result, the relevant datapoints and their corresponding values in the figure in the bottom right corner will be highlighted.</li>
    <li>Pressing the "clear selection" button will reset the figure.</li>
    <li> Additionally, if a data point is hovered, the corresponding disparity output value will be displayed in the bottom right corner.</li>

</ol>

In [8]:
dashboard1

    'data': [{'custo…

### Dashboard 2: Scenes w.r.t. {metric} with color coded "epochs"
An "epoch" in this context means an experiment with the same settings evaluated across every scene in the Middlebury 2004 training dataset.<br>
<ol>
    <li>The following figure allows to turn all the plots on and off</li>
    <li>Additionally, their visibiliy can also be handled by interacting with their legend entries on the right side of the plot.
    </li>
    <li> Therefore custom comparison can be made between different scenes, kernel sizes and match values. </li>
    <li> The figure in the bottom left corner shows the corresponding disparity map. </li>
    <li> The figure in the bottom right corner shows the corresponding ground truth disparity map. </li>
</ol>

In [9]:
dashboard2

### Dashboard 3: "Epoch" w.r.t. {metric} with color coded scenes
An "epoch" in this context means an experiment with the same settings evaluated across every scene in the Middlebury 2004 training dataset.<br>
<ol>
    <li>The following figure allows to turn all the plots on and off</li>
    <li>Additionally, their visibiliy can also be handled by interacting with their legend entries on the right side of the plot.
    </li>
    <li> Therefore custom comparison can be made between different scenes, kernel sizes and match values. </li>
    <li> The figure in the bottom left corner shows the corresponding disparity map. </li>
    <li> The figure in the bottom right corner shows the corresponding ground truth disparity map. </li>
</ol>

In [10]:
dashboard3